In [37]:
from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_from_json(json_data):
    queries = []

    # Iterate through the items in the JSON object
    for item_key, item_value in json_data.items():
        if not item_value:
            continue

        # Replace problematic characters in keys for node labels
        node_label = item_key.replace("Ifc", "Ifc_").replace('.', '_')
        parent_label = item_value.get("Parent", "").replace("Ifc", "Ifc_").replace('.', '_')

        # Start building the query for the component node
        component_query = f"MERGE (component:{node_label} {{name: $component_name"

        # Initialize a dictionary for parameters
        params = {'component_name': node_label}

        # Add properties to the component node from the JSON
        for pset_name, pset_details in item_value.get("Psets", {}).items():
            for prop_name, prop_details in pset_details.get("Properties", {}).items():
                # Add property to the query
                safe_prop_name = prop_name.replace(' ', '_')  
                component_query += f", {safe_prop_name}: ${safe_prop_name}"
                params[safe_prop_name] = " "

        # Close the curly braces for the MERGE statement
        component_query += "})"

        # If a parent label exists, add relationship queries
        if parent_label:
            parent_query = (
                f" MERGE (parent:{parent_label} {{name: $parent_name}})"
                " MERGE (component)-[:IS_A]->(parent)"
            )
            params['parent_name'] = parent_label
            component_query += parent_query

        # Add the completed query and parameters to the list
        queries.append({'query': component_query, 'params': params})

    return queries

import json
import pandas as pd

# Assuming you have a JSON file named 'data.json'
with open('IFC.json', 'r') as file:
    data = json.load(file)

queries= create_cypher_queries_from_json(data["Domain"]["Classifications"])

#run_cypher_query(driver, " ".join(queries))
def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries:
    run_cypher_query(driver, query_info['query'], query_info['params'])


In [ ]:
from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_from_json(json_data):
    queries = []

    # Iterate through the items in the JSON object
    for item_key, item_value in json_data.items():
        if not item_value:
            continue

        # Replace problematic characters in keys for node labels
        node_label = item_key.replace("Ifc", "Ifc_").replace('.', '_')
        parent_label = item_value.get("Parent", "").replace("Ifc", "Ifc_").replace('.', '_')

        # Start building the query for the component node
        component_query = f"MERGE (component:{node_label} {{name: $component_name"

        # Initialize a dictionary for parameters
        params = {'component_name': node_label}

        # Add properties to the component node from the JSON
        for pset_name, pset_details in item_value.get("Psets", {}).items():
            for prop_name, prop_details in pset_details.get("Properties", {}).items():
                # Add property to the query
                safe_prop_name = prop_name.replace(' ', '_')  
                component_query += f", {safe_prop_name}: ${safe_prop_name}"
                params[safe_prop_name] = " "

        # Close the curly braces for the MERGE statement
        component_query += "})"

        # If a parent label exists, add relationship queries
        if parent_label:
            parent_query = (
                f" MERGE (parent:{parent_label} {{name: $parent_name}})"
                " MERGE (component)-[:IS_A]->(parent)"
            )
            params['parent_name'] = parent_label
            component_query += parent_query

        # Add the completed query and parameters to the list
        queries.append({'query': component_query, 'params': params})

    return queries

import json
import pandas as pd

# Assuming you have a JSON file named 'data.json'
with open('IFC.json', 'r') as file:
    data = json.load(file)

queries= create_cypher_queries_from_json(data["Domain"]["Classifications"])

#run_cypher_query(driver, " ".join(queries))
def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries:
    run_cypher_query(driver, query_info['query'], query_info['params'])


In [9]:
import json
import pandas as pd
# Assuming you have a JSON file named 'data.json'
with open('IFC.json', 'r') as file:
    data = json.load(file)

item_key_list = []
pset_name_list=[]
prop_name_list=[]
prop_details_list=[]
for item_key, item_value in data["Domain"]["Classifications"].items():
    # Add properties and property sets
    for pset_name, pset_details in item_value.get("Psets", {}).items():
        for prop_name, prop_details in pset_details.get("Properties", {}).items():
            if prop_details["type"] == "string" or prop_details["type"] == "real":
                prop_details = prop_details["type"]
            elif "values" in prop_details.keys():
                prop_details = str(prop_details["values"])
            else:
                prop_details = str(prop_details)

            item_key_list.append(item_key)
            prop_name_list.append(prop_name)
            prop_details_list.append(prop_details)

In [10]:
table = pd.DataFrame({
    "item_key": item_key_list,
    "prop_name": prop_name_list,
    "prop_details": prop_details_list
})

In [11]:
table[0:10]

,item_key,prop_name,prop_details
0,IfcActuator,ActuatorApplication,"['DAMPERACTUATOR', 'ENTRYEXITDEVICE', 'FIRESMO..."
1,IfcActuator,ActuatorStatus,"['DEMOLISH', 'EXISTING', 'NEW', 'NOTKNOWN', 'O..."
2,IfcActuator,FailPosition,"['FAILCLOSED', 'FAILOPEN', 'NOTKNOWN', 'OTHER'..."
3,IfcActuator,ManualOverride,"['TRUE', 'FALSE']"
4,IfcActuator,Reference,string
5,IfcActuator,Force,real
6,IfcActuator,Stroke,real
7,IfcActuator,RangeAngle,real
8,IfcActuator,Torque,real
9,IfcActuator,GrossWeight,real


In [ ]:
uni = pd.read_csv("Uniclass2015.csv")
uni

,Code,Group,Sub group,Section,Object,Title,NBS Code,NRM
0,Ac,NaN,NaN,NaN,NaN,Activities,NaN,NaN
1,Co,NaN,NaN,NaN,NaN,Complexes,NaN,NaN
2,EF,NaN,NaN,NaN,NaN,Elements/ functions,NaN,NaN
3,En,NaN,NaN,NaN,NaN,Entities,NaN,NaN
4,FI,NaN,NaN,NaN,NaN,Form of Information,NaN,NaN
...,...,...,...,...,...,...,...,...
15370,Ro_70_70_37,70.0,70.0,37.0,NaN,Highways operative,NaN,NaN
15371,Ro_70_70_68,70.0,70.0,68.0,NaN,Rail labourer,NaN,NaN
15372,Ro_70_70_70,70.0,70.0,70.0,NaN,Rail track layer,NaN,NaN
15373,Ro_70_70_72,70.0,70.0,72.0,NaN,Rail traction technician,NaN,NaN


In [36]:

from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_mf(mf):
    queries = []
    for index, row in mf.iterrows():
        # Clean up the code to be a valid Cypher label by replacing spaces with underscores and prefixing with a letter
        component_label = 'C' + row['code'].replace(' ', '_').replace('.', '_')
        parent_label = 'C' + row['parent'].replace(' ', '_').replace('.', '_') if row['parent'] != 'None' else False 
        c_label = 'C' + row['label']

        # Create the MERGE queries
        component_query = f"MERGE (component:{component_label} {{name: $component_label, label:$c_label}})"
        params = {'component_label': component_label, 'c_label': c_label}

        if parent_label:
            parent_query = (
                f" MERGE (parent:{parent_label} {{name: $parent_label}})"
                " MERGE (component)-[:IS_A]->(parent)"
            )
            params['parent_label'] = parent_label
        
            component_query += parent_query
        
        # Parameters for the query
        
        # Add the query and params to the list
        queries.append({'query': component_query, 'params': params})
        
    return queries
mf = pd.read_csv("MF_processed.csv")

queries= create_cypher_queries_mf(mf)

def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries[7139:]:
    run_cypher_query(driver, query_info['query'], query_info['params'])

In [39]:
eBKP = pd.read_csv("eBKP_processed.csv")
eBKP

,Unnamed: 0,raw,code,word,Translated_Text
0,0,A01.01 Grundstückserwerb;A01.01;Grundstückserw...,A01.01,Grundstückserwerb,Property acquisition
1,1,A01.02 Baurechtserwerb;A01.02;Baurechtserwerb;...,A01.02,Baurechtserwerb,Building law acquisition
2,2,A01.03 Bauwerkserwerb;A01.03;Bauwerkserwerb;;;...,A01.03,Bauwerkserwerb,Building acquisition
3,3,A01.04 Perimeterbeitrag;A01.04;Perimeterbeitra...,A01.04,Perimeterbeitrag,Perimeter contribution
4,4,A02.01 Handänderungssteuer,A02.01,Handänderungssteuer,Hand change tax
...,...,...,...,...,...
323,323,W07.01 Betriebsaufwand;W07.01;Betriebsaufwand;...,W07.01,Betriebsaufwand,Operating effort
324,324,W07.02 Betriebsertrag;W07.02;Betriebsertrag;;;...,W07.02,Betriebsertrag,Operating income
325,325,Y01.01 Reserve;Y01.01;Reserve;;;;;;Reserve,Y01.01,Reserve,reserve
326,326,Y02.01 Teuerung;Y02.01;Teuerung;;;;;;Teuerung,Y02.01,Teuerung,Inflation


In [40]:

from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_eBKP(df):
    queries = []
    for index, row in df.iterrows():
        # Clean up the code to be a valid Cypher label by replacing spaces with underscores and prefixing with a letter
        component_label = 'B_' + row['code'].replace(' ', '_').replace('.', '_')
        c_label = 'B_' + row['Translated_Text']

        # Create the MERGE queries
        component_query = f"MERGE (component:{component_label} {{name: $component_label, label:$c_label}})"
        params = {'component_label': component_label, 'c_label': c_label}
        
        
        # Parameters for the query
        
        # Add the query and params to the list
        queries.append({'query': component_query, 'params': params})
        
    return queries

eBKP = pd.read_csv("eBKP_processed.csv")

queries= create_cypher_queries_eBKP(eBKP)

def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries:
    run_cypher_query(driver, query_info['query'], query_info['params'])